In [1]:
import pandas as pd
import numpy as np
import keras.backend as K
from keras.utils import get_custom_objects
from keras.layers import Layer, Input, Conv2D, Conv3D, Reshape, BatchNormalization, Activation, Dropout
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, roc_auc_score, roc_curve
import statsmodels.api as sm
from math import sqrt
from tqdm.notebook import tqdm
import seaborn as sns

Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5498903038364008141
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6966896648802742306
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10148974707430616801
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 630268649388154450
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 644755265186958750
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:3"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10018984676128151756
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
def division(data):
    '''
    train:validation:test=6:2:2 데이터 분할
    '''
    train_size = int(len(data)*0.6)
    val_size = int(len(data)*0.8)
    
    data_train = data[0:train_size]
    data_val = data[train_size:val_size]
    data_test = data[val_size:len(data)]
    
    return data_train, data_val, data_test


def look_back(X, Y, a):
    '''
    시계열 분석을 위해 timestep과 lag를 적용하여 데이터셋을 생성
    '''
    X_lb = np.zeros((len(X) - a, a, 40, 28, 11))
    for l in range(len(X) - a): 
        for r in range(a):
            X_lb[l, r] = X[l + r, 0]
    Y = Y[a:]
    
    return X_lb, Y


def get_dropout(input_tensor, p=0.5, mc=False):
    '''
    몬테카를로 드롭아웃
    기존 학습 과정에서만 적용되었던 드롭아웃을 테스트 과정에서도 적용되도록 한다.
    '''
    if mc:
        return Dropout(p)(input_tensor, training=True)
    else:
        return Dropout(p)(input_tensor)

    
# 커스텀 객체 선언
# 참고 : https://hwiyong.tistory.com/375
def mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': mish})

In [5]:
def ICNN_MCDO(time_st):
    '''
    기존 ICNN모델에 Monte Carlo Dropout과 L2 regularization을 적용하여 베이지안 뉴럴 네트워크에 근사한 값을 출력한다.
    또한 테스트 단계에서 Monte Carlo Sampling을 통해 예측값의 불확실성을 추정한다.
    
    
    적용 순서
    Convolution - Batch Normalization - Activation - Dropout - Pooling
    
    참고 : https://gaussian37.github.io/dl-concept-order_of_regularization_term/
    '''
    
    input_tensor = Input(shape=(time_st, 40, 28, 11))
    x = Conv3D(64, (time_st, 1, 1), padding='valid', kernel_regularizer=l2(0.01))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25, mc=True)
    
    x = Reshape((40, 28, 64))(x)
    
    x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25, mc=True)
    
    x = Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25, mc=True)
    
    x = Conv2D(16, (3, 3), padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25, mc=True)
    
    x = Conv2D(8, (3, 3), padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25, mc=True)
    
    output_tensor = Conv2D(1, (3, 3), padding='same')(x)

    model = Model(input_tensor, output_tensor)

    return model

In [6]:
def ICNN_Dropout(time_st):
    input_tensor = Input(shape=(time_st, 40, 28, 11))
    x = Conv3D(64, (time_st, 1, 1), padding='valid')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25)
    
    x = Reshape((40, 28, 64))(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25)
    
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25)
    
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25)
    
    x = Conv2D(8, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    x = get_dropout(x, p=0.25)
    
    output_tensor = Conv2D(1, (3, 3), padding='same')(x)

    model = Model(input_tensor, output_tensor)

    return model

In [7]:
def ICNN(time_st):
    input_tensor = Input(shape=(time_st, 40, 28, 11))
    x = Conv3D(64, (time_st, 1, 1), padding='valid')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    
    x = Reshape((40, 28, 64))(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    
    x = Conv2D(8, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(mish)(x)
    
    output_tensor = Conv2D(1, (3, 3), padding='same')(x)

    model = Model(input_tensor, output_tensor)

    return model

In [8]:
'''
디렉토리에서 데이터를 불러온후 Reshape를 통해 이미지 형식으로 변환
'''

path_dir = "./IDWDATA_40_28size/"
file_list = ['CO_sizereset.csv','NO2_sizereset.csv', 'O3_sizereset.csv', 'PM10_sizereset.csv','PM25_sizereset.csv', 'SO2_sizereset.csv', 'temp_sizereset.csv', 'wind_x_sizereset.csv','wind_y_sizereset.csv','windspeed_sizereset.csv','rainfall_sizereset.csv']

concat = []
dataset = []
flg = 0
i = 0
for f in file_list:
    df = pd.read_csv(path_dir + f, engine='python')
    del df['Unnamed: 0']

    df_flat = df.values.flatten()
    df_flat_rs = df_flat.reshape(len(df_flat), 1)

    if (i == 0):
        concat = df_flat_rs
    else:
        concat = np.concatenate((concat, df_flat_rs), axis=1)
    i = i+1

if(flg==0):
    dataset = concat
else:
    dataset = np.concatenate((dataset, concat), axis=0)
flg = flg+1

# reshape here
data = np.reshape(dataset, ((8760*2), 1, 40, 28, 11))

### 1) ICNN-MCDO : PM10

In [14]:
import math
for time_step in [24]:
    for lag in [24]:
        print('########## Currnent time step:{0}, lag:{1} ##########'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, mode='min')
        mc = ModelCheckpoint('./Result/PM10/ICNN-MCDO/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

 
        
        #model = ICNN_MCDO(time_step)
        model = load_model('./Result/PM10/ICNN-MCDO/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)))
    
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=300, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

########## Currnent time step:24, lag:24 ##########


KeyboardInterrupt: 

In [ ]:
for time_step in tqdm([1, 12, 24]):
    for lag in tqdm([1, 2, 4, 6, 12, 24]):
        model_src = './Result/PM10/ICNN-MCDO/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm10 = np.zeros((100, Y_test.shape[0],40, 28))
        
        for i in range(100):
            print(i)
            tmp = model.predict(X_test)
            tmp = tmp.reshape(-1, 40, 28)
            Y_pred_pm10[i, :, :, :] = tmp
        
        np.save('./Result/PM10/ICNN-MCDO/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm10)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm10

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46

  0%|          | 0/6 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46

  0%|          | 0/6 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


### 2) ICNN-Dropout : PM10

In [ ]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        print('Currnent time step:{0}, lag:{1}'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, mode='min')
        mc = ModelCheckpoint('./Result/PM10/ICNN-Dropout/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

        X_data = data
        Y_data = data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        #model = ICNN_Dropout(time_step)
        model = load_model('./Result/PM10/ICNN-Dropout/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)))

        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=300, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])

        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

In [ ]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        model_src = './Result/PM10/ICNN-Dropout/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm10 = model.predict(X_test).reshape(-1, 40, 28)
        
        np.save('./Result/PM10/ICNN-Dropout/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm10)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm10

### 3) ICNN : PM10

In [9]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        print('Currnent time step:{0}, lag:{1}'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='min')
        mc = ModelCheckpoint('./Result/PM10/ICNN/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

        X_data = data
        Y_data = data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        model = ICNN(time_step)

        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=2000, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])

        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

Currnent time step:24, lag:6
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 10494 samples, validate on 3498 samples
Epoch 1/2000
10494/10494 [==============================] - 91s 9ms/step - loss: 2578.0025 - val_loss: 558.4317
Epoch 2/2000
10494/10494 [==============================] - 91s 9ms/step - loss: 2493.8127 - val_loss: 917.8928
Epoch 3/2000
10494/10494 [==============================] - 93s 9ms/step - loss: 2429.2595 - val_loss: 1094.3077
Epoch 4/2000
10494/10494 [==============================] - 92s 9ms/step - loss: 2359.4482 - val_loss: 1183.8533
Epoch 5/2000
10494/10494 [==============================] - 93s 9ms/step - loss: 2286.4050 - val_loss: 1259.6314
Epoch 6/2000
10494/10494 [==============================] - 94s 9ms/step - loss: 2206.9321 - val_loss: 1361.2951
Epoch 7/2000
10494/10494 [==============================] - 94s 9ms/step - loss: 2123.6694 - val_loss: 1281.3749
Epoch 8/2000
10494/10494 [=========================

Epoch 46/2000
10491/10491 [==============================] - 92s 9ms/step - loss: 167.5642 - val_loss: 215.2747
Epoch 47/2000
10491/10491 [==============================] - 96s 9ms/step - loss: 164.9965 - val_loss: 223.7073
Epoch 48/2000
10491/10491 [==============================] - 94s 9ms/step - loss: 160.1261 - val_loss: 202.6296
Epoch 49/2000
10491/10491 [==============================] - 93s 9ms/step - loss: 154.3854 - val_loss: 203.5759
Epoch 50/2000
10491/10491 [==============================] - 94s 9ms/step - loss: 154.5538 - val_loss: 214.3516
Epoch 51/2000
10491/10491 [==============================] - 94s 9ms/step - loss: 153.4172 - val_loss: 212.2157
Epoch 52/2000
10491/10491 [==============================] - 92s 9ms/step - loss: 149.3138 - val_loss: 198.9901
Epoch 53/2000
10491/10491 [==============================] - 94s 9ms/step - loss: 148.3130 - val_loss: 200.6796
Epoch 54/2000
10491/10491 [==============================] - 95s 9ms/step - loss: 146.9849 - val_loss: 2

In [10]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        model_src = './Result/PM10/ICNN/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,3] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm10 = model.predict(X_test).reshape(-1, 40, 28)
        
        np.save('./Result/PM10/ICNN/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm10)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm10

### 4) ICNN-MCDO : PM2.5

In [11]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        print('Currnent time step:{0}, lag:{1}'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='min')
        mc = ModelCheckpoint('./Result/PM2.5/ICNN-MCDO/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        model = ICNN_MCDO(time_step)

        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=2000, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])

        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

Currnent time step:1, lag:1
Train on 10511 samples, validate on 3504 samples
Epoch 1/2000
10511/10511 [==============================] - 64s 6ms/step - loss: 861.5862 - val_loss: 1336.6135
Epoch 2/2000
10511/10511 [==============================] - 61s 6ms/step - loss: 814.1716 - val_loss: 490.4784
Epoch 3/2000
10511/10511 [==============================] - 61s 6ms/step - loss: 769.9417 - val_loss: 90.8664
Epoch 4/2000
10511/10511 [==============================] - 59s 6ms/step - loss: 715.3002 - val_loss: 94.4531
Epoch 5/2000
10511/10511 [==============================] - 62s 6ms/step - loss: 658.8618 - val_loss: 142.0728
Epoch 6/2000
10511/10511 [==============================] - 61s 6ms/step - loss: 604.0180 - val_loss: 187.3913
Epoch 7/2000
10511/10511 [==============================] - 61s 6ms/step - loss: 549.9644 - val_loss: 213.0637
Epoch 8/2000
10511/10511 [==============================] - 61s 6ms/step - loss: 497.9778 - val_loss: 210.4455
Epoch 9/2000
10511/10511 [==========

10508/10508 [==============================] - 60s 6ms/step - loss: 66.6705 - val_loss: 51.1610
Epoch 78/2000
10508/10508 [==============================] - 61s 6ms/step - loss: 66.6100 - val_loss: 48.7250
Epoch 79/2000
10508/10508 [==============================] - 62s 6ms/step - loss: 66.6444 - val_loss: 51.2825
Epoch 80/2000
10508/10508 [==============================] - 61s 6ms/step - loss: 66.3629 - val_loss: 51.6094
Epoch 81/2000
10508/10508 [==============================] - 61s 6ms/step - loss: 65.2832 - val_loss: 49.3588
Epoch 82/2000
10508/10508 [==============================] - 60s 6ms/step - loss: 65.0016 - val_loss: 48.1962
Epoch 83/2000
10508/10508 [==============================] - 61s 6ms/step - loss: 64.7997 - val_loss: 49.4947
Epoch 84/2000
10508/10508 [==============================] - 62s 6ms/step - loss: 65.6699 - val_loss: 49.5069
Epoch 85/2000
10508/10508 [==============================] - 61s 6ms/step - loss: 65.0816 - val_loss: 48.4492
Epoch 86/2000
10508/1050

10497/10497 [==============================] - 59s 6ms/step - loss: 156.8581 - val_loss: 114.7490
Epoch 42/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 157.8583 - val_loss: 112.1866
Epoch 43/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 157.5832 - val_loss: 117.2430
Epoch 44/2000
10497/10497 [==============================] - 60s 6ms/step - loss: 156.7311 - val_loss: 111.4892
Epoch 45/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 159.1367 - val_loss: 116.5031
Epoch 46/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 158.9600 - val_loss: 118.0008
Epoch 47/2000
10497/10497 [==============================] - 60s 6ms/step - loss: 155.5195 - val_loss: 114.7882
Epoch 48/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 157.2328 - val_loss: 125.7619
Epoch 49/2000
10497/10497 [==============================] - 59s 6ms/step - loss: 156.2715 - val_loss: 114.1389
Epoch 

10504/10504 [==============================] - 83s 8ms/step - loss: 15.4791 - val_loss: 11.5501
Epoch 125/2000
10504/10504 [==============================] - 82s 8ms/step - loss: 15.3290 - val_loss: 10.6963
Epoch 126/2000
10504/10504 [==============================] - 82s 8ms/step - loss: 14.7362 - val_loss: 10.8188
Currnent time step:12, lag:2
Train on 10504 samples, validate on 3501 samples
Epoch 1/2000
10504/10504 [==============================] - 86s 8ms/step - loss: 885.5298 - val_loss: 144.2300
Epoch 2/2000
10504/10504 [==============================] - 83s 8ms/step - loss: 853.7586 - val_loss: 93.6290
Epoch 3/2000
10504/10504 [==============================] - 82s 8ms/step - loss: 829.7831 - val_loss: 143.2816
Epoch 4/2000
10504/10504 [==============================] - 82s 8ms/step - loss: 793.8055 - val_loss: 173.2243
Epoch 5/2000
10504/10504 [==============================] - 82s 8ms/step - loss: 756.5093 - val_loss: 188.6690
Epoch 6/2000
10504/10504 [========================

10503/10503 [==============================] - 82s 8ms/step - loss: 33.4673 - val_loss: 28.5385
Epoch 123/2000
10503/10503 [==============================] - 84s 8ms/step - loss: 33.4867 - val_loss: 27.0036
Epoch 124/2000
10503/10503 [==============================] - 84s 8ms/step - loss: 33.8704 - val_loss: 27.3909
Epoch 125/2000
10503/10503 [==============================] - 83s 8ms/step - loss: 33.4499 - val_loss: 28.2449
Epoch 126/2000
10503/10503 [==============================] - 84s 8ms/step - loss: 33.4525 - val_loss: 28.9709
Epoch 127/2000
10503/10503 [==============================] - 83s 8ms/step - loss: 34.9039 - val_loss: 26.7718
Epoch 128/2000
10503/10503 [==============================] - 82s 8ms/step - loss: 33.6879 - val_loss: 26.8367
Epoch 129/2000
10503/10503 [==============================] - 83s 8ms/step - loss: 34.1941 - val_loss: 38.9105
Epoch 130/2000
10503/10503 [==============================] - 83s 8ms/step - loss: 35.0103 - val_loss: 27.8373
Epoch 131/2000
1

...

10497/10497 [==============================] - 110s 10ms/step - loss: 272.0615 - val_loss: 169.8567
Epoch 13/2000
10497/10497 [==============================] - 112s 11ms/step - loss: 237.0556 - val_loss: 193.8523
Epoch 14/2000
10497/10497 [==============================] - 108s 10ms/step - loss: 205.0256 - val_loss: 176.2196
Epoch 15/2000
10497/10497 [==============================] - 109s 10ms/step - loss: 177.7011 - val_loss: 188.7412
Epoch 16/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 154.9494 - val_loss: 230.8015
Epoch 17/2000
10497/10497 [==============================] - 111s 11ms/step - loss: 135.4540 - val_loss: 110.0435
Epoch 18/2000
10497/10497 [==============================] - 111s 11ms/step - loss: 116.6199 - val_loss: 163.9212
Epoch 19/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 99.8432 - val_loss: 138.2038
Epoch 20/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 82.8628 - val_loss:

10497/10497 [==============================] - 110s 10ms/step - loss: 21.3651 - val_loss: 15.0811
Epoch 86/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 21.4836 - val_loss: 17.4337
Epoch 87/2000
10497/10497 [==============================] - 109s 10ms/step - loss: 21.7378 - val_loss: 15.4853
Epoch 88/2000
10497/10497 [==============================] - 107s 10ms/step - loss: 21.2606 - val_loss: 15.2410
Epoch 89/2000
10497/10497 [==============================] - 111s 11ms/step - loss: 21.6921 - val_loss: 16.5293
Epoch 90/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 23.1279 - val_loss: 20.4574
Epoch 91/2000
10497/10497 [==============================] - 110s 10ms/step - loss: 21.8109 - val_loss: 18.2855
Epoch 92/2000
10497/10497 [==============================] - 111s 11ms/step - loss: 23.1937 - val_loss: 15.6505
Epoch 93/2000
10497/10497 [==============================] - 111s 11ms/step - loss: 21.0667 - val_loss: 15.2018
Epoch 

10495/10495 [==============================] - 168s 16ms/step - loss: 715.4779 - val_loss: 250.1819
Epoch 6/2000
10495/10495 [==============================] - 167s 16ms/step - loss: 665.6256 - val_loss: 343.1776
Epoch 7/2000
10495/10495 [==============================] - 168s 16ms/step - loss: 615.3586 - val_loss: 387.4507
Epoch 8/2000
10495/10495 [==============================] - 168s 16ms/step - loss: 565.3900 - val_loss: 259.3810
Epoch 9/2000
10495/10495 [==============================] - 167s 16ms/step - loss: 515.5478 - val_loss: 307.1666
Epoch 10/2000
10495/10495 [==============================] - 167s 16ms/step - loss: 465.1023 - val_loss: 290.7053
Epoch 11/2000
10495/10495 [==============================] - 165s 16ms/step - loss: 417.4309 - val_loss: 351.7048
Epoch 12/2000
10495/10495 [==============================] - 167s 16ms/step - loss: 371.8134 - val_loss: 243.4045
Epoch 13/2000
10495/10495 [==============================] - 167s 16ms/step - loss: 329.7925 - val_loss: 2

Epoch 77/2000
10494/10494 [==============================] - 123s 12ms/step - loss: 44.8348 - val_loss: 40.9226
Epoch 78/2000
10494/10494 [==============================] - 125s 12ms/step - loss: 44.0679 - val_loss: 41.3791
Epoch 79/2000
10494/10494 [==============================] - 127s 12ms/step - loss: 44.2060 - val_loss: 45.1991
Epoch 80/2000
10494/10494 [==============================] - 128s 12ms/step - loss: 44.6813 - val_loss: 40.6306
Epoch 81/2000
10494/10494 [==============================] - 125s 12ms/step - loss: 42.7744 - val_loss: 41.5065
Epoch 82/2000
10494/10494 [==============================] - 122s 12ms/step - loss: 43.0324 - val_loss: 43.9552
Epoch 83/2000
10494/10494 [==============================] - 124s 12ms/step - loss: 43.2275 - val_loss: 45.9822
Epoch 84/2000
10494/10494 [==============================] - 123s 12ms/step - loss: 42.6772 - val_loss: 46.0189
Epoch 85/2000
10494/10494 [==============================] - 124s 12ms/step - loss: 43.3995 - val_loss: 

Epoch 49/2000
10491/10491 [==============================] - 104s 10ms/step - loss: 81.0712 - val_loss: 90.5323
Epoch 50/2000
10491/10491 [==============================] - 106s 10ms/step - loss: 80.1359 - val_loss: 87.9525
Epoch 51/2000
10491/10491 [==============================] - 104s 10ms/step - loss: 79.0344 - val_loss: 79.5028
Epoch 52/2000
10491/10491 [==============================] - 105s 10ms/step - loss: 81.2202 - val_loss: 80.1032
Epoch 53/2000
10491/10491 [==============================] - 105s 10ms/step - loss: 79.4570 - val_loss: 88.3464
Epoch 54/2000
10491/10491 [==============================] - 105s 10ms/step - loss: 77.5120 - val_loss: 79.8196
Epoch 55/2000
10491/10491 [==============================] - 105s 10ms/step - loss: 76.0838 - val_loss: 79.6849
Currnent time step:24, lag:24
Train on 10483 samples, validate on 3495 samples
Epoch 1/2000
10483/10483 [==============================] - 111s 11ms/step - loss: 879.0686 - val_loss: 445.0136
Epoch 2/2000
10483/10483

In [8]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        model_src = './Result/PM2.5/ICNN-MCDO/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm25 = np.zeros((100, Y_test.shape[0],40, 28))
        
        for i in range(100):
            tmp = model.predict(X_test)
            tmp = tmp.reshape(-1, 40, 28)
            Y_pred_pm25[i, :, :, :] = tmp
        
        np.save('./Result/PM2.5/ICNN-MCDO/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm25)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm25

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



### 5) ICNN-Dropout : PM2.5

In [ ]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        print('Currnent time step:{0}, lag:{1}'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='min')
        mc = ModelCheckpoint('./Result/PM2.5/ICNN-Dropout/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        model = ICNN_Dropout(time_step)

        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=2000, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])

        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

In [ ]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        model_src = './Result/PM2.5/ICNN-Dropout/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm25 = model.predict(X_test).reshape(-1, 40, 28)
        
        np.save('./Result/PM2.5/ICNN-Dropout/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm25)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm25

### 6) ICNN : PM2.5

In [9]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        print('Currnent time step:{0}, lag:{1}'.format(str(time_step), str(lag)))

        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='min')
        mc = ModelCheckpoint('./Result/PM2.5/ICNN/Model/timestep{0}_lag{1}.h5'.format(str(time_step), str(lag)), monitor='val_loss', mode='min', save_best_only=True)

        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        model = ICNN(time_step)

        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=2000, batch_size=512, validation_data=(X_val, Y_val), callbacks=[early_stopping, mc])

        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test

Currnent time step:1, lag:1
Train on 10511 samples, validate on 3504 samples
Epoch 1/2000
10511/10511 [==============================] - 52s 5ms/step - loss: 891.3173 - val_loss: 227.0962
Epoch 2/2000
10511/10511 [==============================] - 50s 5ms/step - loss: 847.7753 - val_loss: 166.5910
Epoch 3/2000
10511/10511 [==============================] - 51s 5ms/step - loss: 811.2432 - val_loss: 213.1153
Epoch 4/2000
10511/10511 [==============================] - 51s 5ms/step - loss: 773.1734 - val_loss: 268.1833
Epoch 5/2000
10511/10511 [==============================] - 50s 5ms/step - loss: 733.2273 - val_loss: 294.2687
Epoch 6/2000
10511/10511 [==============================] - 51s 5ms/step - loss: 691.7277 - val_loss: 282.2980
Epoch 7/2000
10511/10511 [==============================] - 51s 5ms/step - loss: 647.9768 - val_loss: 267.3409
Epoch 8/2000
10511/10511 [==============================] - 51s 5ms/step - loss: 600.5761 - val_loss: 243.1777
Epoch 9/2000
10511/10511 [=========

10511/10511 [==============================] - 49s 5ms/step - loss: 7.1840 - val_loss: 4.9897
Epoch 76/2000
10511/10511 [==============================] - 48s 5ms/step - loss: 6.9238 - val_loss: 4.5662
Epoch 77/2000
10511/10511 [==============================] - 49s 5ms/step - loss: 6.9240 - val_loss: 4.7237
Epoch 78/2000
10511/10511 [==============================] - 49s 5ms/step - loss: 6.6184 - val_loss: 4.4303
Epoch 79/2000
10511/10511 [==============================] - 48s 5ms/step - loss: 6.5597 - val_loss: 4.4504
Epoch 80/2000
10511/10511 [==============================] - 49s 5ms/step - loss: 6.4865 - val_loss: 4.5492
Epoch 81/2000
10511/10511 [==============================] - 48s 5ms/step - loss: 6.1124 - val_loss: 4.2688
Epoch 82/2000
10511/10511 [==============================] - 48s 5ms/step - loss: 6.6597 - val_loss: 4.5125
Epoch 83/2000
10511/10511 [==============================] - 50s 5ms/step - loss: 6.8944 - val_loss: 4.6637
Epoch 84/2000
10511/10511 [===============

Epoch 3/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 781.2723 - val_loss: 196.4615
Epoch 4/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 724.8336 - val_loss: 199.0042
Epoch 5/2000
10510/10510 [==============================] - 50s 5ms/step - loss: 669.0435 - val_loss: 189.7806
Epoch 6/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 614.1861 - val_loss: 185.2826
Epoch 7/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 559.8756 - val_loss: 194.9053
Epoch 8/2000
10510/10510 [==============================] - 52s 5ms/step - loss: 506.5583 - val_loss: 204.4629
Epoch 9/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 455.2544 - val_loss: 208.3913
Epoch 10/2000
10510/10510 [==============================] - 52s 5ms/step - loss: 405.0406 - val_loss: 196.4585
Epoch 11/2000
10510/10510 [==============================] - 51s 5ms/step - loss: 357.8384 - val_loss: 219.5232

KeyboardInterrupt: 

In [ ]:
for time_step in [1, 12, 24]:
    for lag in [1, 2, 4, 6, 12, 24]:
        model_src = './Result/PM2.5/ICNN/Model/timestep{0}_lag{1}.h5'.format(time_step, lag)
        model = load_model(model_src, custom_objects={'mish':mish})
        
        X_data = data
        Y_data = data[:,:,:,:,4] # 3 = PM10, 4 = PM2.5
        Y_data = Y_data.reshape((8760*2),1,40,28,1)

        X_data, Y_data = look_back(X_data, Y_data, time_step)
        if lag > 1:
            X_data = X_data[0:len(X_data)-(lag-1)]
            Y_data = Y_data[(lag-1):]

        X_train, X_val, X_test = division(X_data)
        Y_train, Y_val, Y_test = division(Y_data)
        Y_train = Y_train.reshape(Y_train.shape[0], 40, 28, 1).astype('float32')
        Y_val = Y_val.reshape(Y_val.shape[0],  40, 28, 1).astype('float32')
        Y_test = Y_test.reshape(Y_test.shape[0], 40, 28, 1).astype('float32')
        
        Y_pred_pm25 = model.predict(X_test).reshape(-1, 40, 28)
        
        np.save('./Result/PM2.5/ICNN/Predict/timestep{0}_lag{1}'.format(time_step, lag), Y_pred_pm25)
        
        del model, X_data, Y_data, X_train, X_val, X_test, Y_train, Y_val, Y_test, Y_pred_pm25